In [1]:
import math
import matplotlib.pyplot as plt
import numpy as np
from pycocotools.coco import COCO
import random
import tensorflow as tf
from tqdm import tqdm

TRANSFER = True

In [2]:
# https://cs230.stanford.edu/blog/datapipeline/

# Create a list of filenames and labels for the dataset
def create_pairs(location = '/home/gregory/Datasets/COCO/2017', mode = 'val'):
    
    # Each 'label' vector is large enough for easy indexing, but this means it contains unused indices
    dim = 91
    
    file = '{}/annotations/instances_{}2017.json'.format(location, mode)
    
    coco = COCO(file)
    
    images = coco.loadImgs(coco.getImgIds())
    
    filenames = []
    labels = []
    
    for i in range(len(images)):
    
        im_obj = images[i]
        
        filenames.append('{}/{}2017/{}'.format(location, mode, im_obj['file_name']))
        
        annotations = coco.loadAnns(coco.getAnnIds(im_obj['id'], iscrowd=None))
        label = np.zeros((dim), dtype = np.float32)
        for x in annotations:
            label[x['category_id']] = 1.0
        labels.append(label)
        
    filenames = np.array(filenames)
    labels = np.array(labels)
        
    return filenames, labels
    
def parse_function(filename, label):
    image_string = tf.io.read_file(filename)

    #Don't use tf.image.decode_image, or the output shape will be undefined
    image = tf.image.decode_jpeg(image_string, channels=3)

    #This will convert to float values in [0, 1]
    image = tf.image.convert_image_dtype(image, tf.float32)

    image = tf.image.resize_with_pad(image, 224, 224)
    
    return image, label

def create_dataset(filenames, labels):

    dataset = tf.data.Dataset.from_tensor_slices((filenames, labels))
    dataset = dataset.shuffle(len(filenames))
    dataset = dataset.map(parse_function, num_parallel_calls=num_parallel_calls)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(1)

    return dataset

In [3]:
# Load a pre-trained Resnet model

model_base = tf.keras.applications.ResNet50(include_top= True, weights='imagenet', input_tensor=None, input_shape = (224, 224, 3), pooling= "avg")

model_base.trainable = False

model = tf.keras.Sequential([
  model_base,
  tf.keras.layers.Dense(91)
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 1000)              25636712  
_________________________________________________________________
dense (Dense)                (None, 91)                91091     
Total params: 25,727,803
Trainable params: 91,091
Non-trainable params: 25,636,712
_________________________________________________________________


In [4]:
model_path = './model_transfer/'

if TRANSFER: 
    learning_rate = 0.001
    learning_rate_decay = 0.3
    learning_rate_drops = 3
    min_epochs = 2
    stopping_epochs = 2
    stopping_tol = 0.001
    batch_size = 64

    folder = '/home/gregory/Datasets/COCO/2017'
    num_parallel_calls = 4

    f, l = create_pairs(mode = 'train')
    n_train = len(f)

    data_train = create_dataset(f, l)

    f, l = create_pairs(mode = 'val')
    n_val = len(f)

    data_val = create_dataset(f, l)

    n_batches_train = math.floor(n_train / batch_size)
    n_batches_val = math.floor(n_val / batch_size)

    def loss(model, inputs, targets):
        preds = model(inputs)
        return tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels = targets, logits = preds))

    def grad(model, inputs, targets):
        with tf.GradientTape() as tape:
            loss_value = loss(model, inputs, targets)
        return loss_value, tape.gradient(loss_value, model.trainable_variables)

    # Setup the initial optimizer (it will be re-initialized when the learning rate gets dropped)
    optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate)

    # Basic counters for the training process
    epoch = 0
    best_epoch = 0
    best_loss = np.inf
    drops = 0

    # Run the training loop
    while True:

        # Check the stopping condition
        if epoch - best_epoch > stopping_epochs and epoch > min_epochs:
            # We have finished only if we have fully decayed the learning_rate
            if drops == learning_rate_drops:
                break
            else:
                print("Dropping learning_rate")
                learning_rate *= learning_rate_decay
                optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate)
                drops += 1

        # Train for an epoch
        epoch_loss_avg = tf.keras.metrics.Mean()
        for (x_batch, y_batch) in tqdm(data_train):
            loss_value, grads = grad(model, x_batch, y_batch)
            optimizer.apply_gradients(zip(grads, model.trainable_variables))
            epoch_loss_avg.update_state(loss_value)
        epoch_loss = epoch_loss_avg.result().numpy()

        # Calculate the validation loss
        epoch_loss_avg_val = tf.keras.metrics.Mean()
        for (x_batch, y_batch) in data_val:
            loss_value = loss(model, x_batch, y_batch)
            epoch_loss_avg_val.update_state(loss_value)
        value = epoch_loss_avg_val.result().numpy()

        # Check if we have made progress
        if value < best_loss - stopping_tol:
            print("Epoch / Epoch Train Loss / Val Loss: " + str(epoch) + " " + str(epoch_loss) + " " + str(value) + " -> saving")
            best_loss = value
            best_epoch = epoch
            model.save_weights(model_path)
        else:
            print("Epoch / Epoch Train Loss / Val Loss: " + str(epoch) + " " + str(epoch_loss) + " " + str(value))

        # Update counters
        epoch += 1

model.load_weights(model_path)


loading annotations into memory...
Done (t=8.86s)
creating index...
index created!
loading annotations into memory...


0it [00:00, ?it/s]

Done (t=0.26s)
creating index...
index created!


1849it [12:09,  2.54it/s]
0it [00:00, ?it/s]

Epoch / Epoch Train Loss / Val Loss: 0 0.31009766 0.16225359 -> saving


...

Epoch / Epoch Train Loss / Val Loss: 8 0.11852247 0.11924724
Dropping learning_rate


1849it [12:09,  2.54it/s]
0it [00:00, ?it/s]

Epoch / Epoch Train Loss / Val Loss: 9 0.11850825 0.11932203
Dropping learning_rate


1849it [12:09,  2.54it/s]


Epoch / Epoch Train Loss / Val Loss: 10 0.11849289 0.11958518
